In [21]:
!pip install gensim


In [41]:
import pandas as pd
import numpy as np 
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from sklearn.metrics import *
from sklearn.feature_extraction import *
from nltk.tokenize import RegexpTokenizer
import re
import string
import random 
from PIL import Image
import requests
from io import BytesIO
from gensim.models import *
import warnings
warnings.filterwarnings(action='ignore',category = UserWarning,module = 'gensim')
warnings.filterwarnings(action = 'ignore',category= FutureWarning,module = 'gensim')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
data =  pd.read_csv("D:/DS Files/netflix_titles.csv")
data.head()


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [24]:
# made a copy of dataset
df = data.copy()
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [25]:
df['title'].count()

8807

In [26]:
df.title[1000]

'Wild Dog'

In [27]:
df1 = pd.DataFrame(df[["title","release_year","description","listed_in"]])

In [28]:
df1

,title,release_year,description,listed_in
0,Dick Johnson Is Dead,2020,"As her father nears the end of his life, filmm...",Documentaries
1,Blood & Water,2021,"After crossing paths at a party, a Cape Town t...","International TV Shows, TV Dramas, TV Mysteries"
2,Ganglands,2021,To protect his family from a powerful drug lor...,"Crime TV Shows, International TV Shows, TV Act..."
3,Jailbirds New Orleans,2021,"Feuds, flirtations and toilet talk go down amo...","Docuseries, Reality TV"
4,Kota Factory,2021,In a city of coaching centers known to train I...,"International TV Shows, Romantic TV Shows, TV ..."
...,...,...,...,...
8802,Zodiac,2007,"A political cartoonist, a crime reporter and a...","Cult Movies, Dramas, Thrillers"
8803,Zombie Dumb,2018,"While living alone in a spooky town, a young g...","Kids' TV, Korean TV Shows, TV Comedies"
8804,Zombieland,2009,Looking to survive in a world taken over by zo...,"Comedies, Horror Movies"
8805,Zoom,2006,"Dragged from civilian life, a former superhero...","Children & Family Movies, Comedies"


In [29]:
df.type.unique()

array(['Movie', 'TV Show'], dtype=object)

In [30]:
# we perform EDA (exploritary data analysis)
# In this we clean the data and find the null values
df.isnull().sum()


show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [31]:
df.dropna()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
8,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...
12,s13,Movie,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic","September 23, 2021",2021,TV-MA,127 min,"Dramas, International Movies",After most of her family is murdered in a terr...
24,s25,Movie,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...",India,"September 21, 2021",1998,TV-14,166 min,"Comedies, International Movies, Romantic Movies",When the father of the man she loves insists t...
...,...,...,...,...,...,...,...,...,...,...,...,...
8801,s8802,Movie,Zinzana,Majid Al Ansari,"Ali Suliman, Saleh Bakri, Yasa, Ali Al-Jabri, ...","United Arab Emirates, Jordan","March 9, 2016",2015,TV-MA,96 min,"Dramas, International Movies, Thrillers",Recovering alcoholic Talal wakes up inside a s...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [32]:
df['description'] = df['description'].astype(str)

In [33]:
des = pd.DataFrame(df['description'])

In [34]:
# creating User Built-in functions
def test(text):
    return "".join(i for i in text if ord(i)<129)
def lower(text):
    return text.lower()

In [39]:
# creating a funtion for stop words
def stop_words(text):
    text = text.split() # split is used to find the stop words
    stop = stopwords.words("english")
    text = [w for w in text if not w in stop]
    return text


In [44]:
def clean_html(text):
    html = re.compile('<.*?>')
    return html.sub(r'',str(text))

In [42]:
def punct(pun):
    token = RegexpTokenizer(r'\w+')
    pun = token.tokenize(pun)
    pun = "".join(pun)
    return pun

In [45]:
df['new_description'] = df['description'].apply(test)
df['new_description'] = df.new_description.apply(lower)
df['new_description'] = df.new_description.apply(func = stop_words)
df['new_description'] = df.new_description.apply(clean_html)
df['new_description'] = df.new_description.apply(punct)

In [47]:
universe = []
for words in df['new_description']:
    universe.append(words.split())